In [ ]:
from pathlib import Path
import getpass
import json
import re
import sys
from typing import List, Optional, Set


def find_root() -> Path:
    for base in [Path.cwd(), *Path.cwd().resolve().parents]:
        if (base / "server.py").exists() and (base / "_index" / "items.json").exists():
            return base

        cand = base / "2026 주정통 코드 정리"
        if (cand / "server.py").exists() and (cand / "_index" / "items.json").exists():
            return cand

    raise RuntimeError(
        "server.py not found. Jupyter를 `2026 주정통 코드 정리` 폴더(또는 하위)에서 실행하세요."
    )


ROOT = find_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from _tools.run_checks_csv import run_checks_to_csv

items = json.loads((ROOT / "_index" / "items.json").read_text(encoding="utf-8"))
unix_items = [it for it in items if str(it.get("code", "")).upper().startswith("U-")]
unix_items.sort(key=lambda it: int(str(it["code"]).split("-", 1)[1]))


def show_items(limit: Optional[int] = None) -> None:
    for idx, it in enumerate(unix_items, start=1):
        print(f"{idx:02d}. {it['code']} ({it['importance']}) {it['section']} - {it['title']}")
        if limit and idx >= limit:
            break


def parse_selection(selection: str) -> List[str]:
    s = (selection or "").strip()
    if not s:
        return []

    s_up = s.upper().replace(" ", "")
    if s_up == "ALL":
        return [str(it["code"]) for it in unix_items]

    out: List[str] = []
    for tok in s_up.split(","):
        if not tok:
            continue

        if re.fullmatch(r"U-\d{2}", tok):
            out.append(tok)
            continue

        if re.fullmatch(r"\d+", tok):
            i = int(tok)
            if 1 <= i <= len(unix_items):
                out.append(str(unix_items[i - 1]["code"]))
            else:
                print(f"무시됨: {tok}")
            continue

        m = re.fullmatch(r"(\d+)-(\d+)", tok)
        if m:
            a, b = int(m.group(1)), int(m.group(2))
            if a > b:
                a, b = b, a
            a = max(1, a)
            b = min(len(unix_items), b)
            if a > b:
                print(f"무시됨: {tok}")
                continue
            for i in range(a, b + 1):
                out.append(str(unix_items[i - 1]["code"]))
            continue

        print(f"무시됨: {tok}")

    dedup: List[str] = []
    seen: Set[str] = set()
    for code in out:
        if code in seen:
            continue
        seen.add(code)
        dedup.append(code)
    return dedup


In [ ]:
show_items()

selection = input("선택(all / 1,2,5-10 / U-01,U-02): ").strip()
codes = parse_selection(selection)
print("선택된 코드:", codes)
if not codes:
    raise RuntimeError("선택된 코드가 없습니다.")

host = input("host(ip/hostname): ").strip()
host_memo = input("host memo(웹/DNS/DB/보안/기타): ").strip()
port_s = input("port[22]: ").strip()
port = int(port_s) if port_s else 22
user = input("user: ").strip()
timeout_s = input("timeout[10]: ").strip()
timeout = int(timeout_s) if timeout_s else 10

key_path = input("key path(없으면 엔터): ").strip()
key = key_path or None
password = None
if not key:
    password = getpass.getpass("password: ")

out_csv = str(ROOT / "_results" / "results.csv")
run_checks_to_csv(
    host=host,
    host_memo=host_memo,
    port=port,
    user=user,
    password=password,
    key=key,
    timeout=timeout,
    codes=codes,
    out=out_csv,
)
print("완료. CSV:", out_csv)
